In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math as m
import os
import datetime
import pandas as pd

In [2]:
path_raw = '//icnas4.cc.ic.ac.uk/rs5115/MSci Project/raw_spectra' #path with raw spectra files
path_logs = '//icnas4.cc.ic.ac.uk/rs5115/MSci Project/bruker_logs' #path with eventlog and temperatures

temp_log = 'thursday2-20180927113645.log'
event_log = 'EventLog_20180927112225.log'

In [24]:
h = 6.626e-34
c = 299792458
k = 1.381e-23

a = 2*h*(c**2)
e = h*c/k

def planck(wavenumber, T): # T in Kelvin
    return 100*a*(wavenumber**3)/(m.exp(e*wavenumber/T)-1)

In [25]:
#RACHEL

In [26]:
def read_wavenumbers(directory, filename):
    lines = open(os.path.join(directory, filename)).readlines() #finds all lines in file as string elements in "lines" list

    wns = []
    for i in lines[5:]:
        line = i.split()
        split_lines = [float(a) for a in line]
        wns.append(split_lines[0])

    return wns

raw_wavenumbers = read_wavenumbers(path_raw, files_list[0])

In [27]:
def read_all_raw_spectra(directory, filename):
    lines = open(os.path.join(directory, filename)).readlines() #finds all lines in file as string elements in "lines" list

    sp1 = []
    sp2 = []
    sp3 = []
    sp4 = []
    
    for i in lines[5:]:
        line = i.split()
        split_lines = [float(a) for a in line]
        sp1.append(split_lines[1])
        sp2.append(split_lines[2])
        sp3.append(split_lines[3])
        sp4.append(split_lines[4])
    
    avg_4_spec = 0.25*(np.array(sp1) + np.array(sp2) + np.array(sp3) + np.array(sp4))
    
    return avg_4_spec #, sp1, sp2, sp3, sp4

In [28]:
average_4_spectra = []        #a list with 352 elements, each of which is a list of 4 spectra, which is a list of 8000 points each
for f in range(len(files_list)-2):
    average_4_spectra.append(read_all_raw_spectra(path_raw, files_list[f]))

In [29]:
spectra_indices = [1,2,71,72,141,142,211,212,281,283,347,348]
wavenums = np.array(raw_wavenumbers)

for spectrum in spectra_indices:       #plot planck curve at some temperature
    temp_avg = final_frame_4.at[spectrum,'temp_avg']
    std = final_frame_4.at[spectrum, 'temp_std']

    radiances_avg = []
    rad_minus = []
    rad_plus = []
    
    for wn in wavenums:
        radiances_avg.append(planck(wn, temp_avg+273.15))
        rad_minus.append(planck(wn, temp_avg+273.15-std))
        rad_plus.append(planck(wn, temp_avg+273.15+std))

    fig = plt.figure()
    plt.plot(wavenums, radiances_avg) #might need to *100 once we have dataframe back
    plt.plot(wavenums, rad_minus)
    plt.plot(wavenums, rad_plus)

    plt.title('Blackbody curve $\pm$ standard deviation for index ' + str(spectrum))
    plt.xlabel('wavenumber $cm^{-1}$')
    plt.ylabel('radiance $W m^{-2} sr^{-1} (cm^{-1})^{-1}$')
    plt.show()

NameError: name 'final_frame_4' is not defined

In [ ]:
#want only 2nd value of hots - averaging over 1min

def transmission_fn(spectrum_number):        #function plotting transmisson of radiance (real/theoretical radiance)
    wavenums = np.array(raw_wavenumbers)
    
    V_h = average_4_spectra[spectrum_number] #first spectrum - make average
    V_c = average_4_spectra[spectrum_number + 1]
    V_diff = np.array(V_h)-np.array(V_c)
    
    temp_average_h = final_frame_4.at[spectrum_number,'temp_avg']
    #std_dev_h = final_frame_4.at[spectrum_number, 'temp_std']
    temp_average_c = final_frame_4.at[spectrum_number+1,'temp_avg']
    #std_dev_c = final_frame_4.at[spectrum_number+1, 'temp_std']
    
    radiances_avg_h = []
    radiances_avg_c = []
    #rad_minus_h = []
    #rad_plus_h = []
    
    for wn in wavenums:
        radiances_avg_h.append(planck(wn, temp_average_h+273.15))
        radiances_avg_c.append(planck(wn, temp_average_c+273.15))
        #rad_minus_h.append(planck(wn, temp_avg_h+273.15-std))
        #rad_plus_h.append(planck(wn, temp_avg_h+273.15+std))
    
    rad_diff = np.array(radiances_avg_h) - np.array(radiances_avg_c)
    
    def division(raw_val,planck_val):
        return raw_val/planck_val
    
    division_vec = np.vectorize(division)
    
    ratio = division_vec(V_diff, rad_diff)
    fig = plt.figure()
    plt.plot(wavenums, ratio)
    plt.xlabel('wavenumber $cm^{-1}$')
    plt.ylabel('response $(W m^{-2} sr^{-1} (cm^{-1})^{-1})^{-1}$ -- fix these units')
    plt.show()

In [ ]:
transmission_fn(3)
transmission_fn(7)
transmission_fn(327)